Generating the Synthetic Dataset:

Defined ranges for each orbital type based on real-world satellite characteristics to generate a Synthetic Dataset

This dataset includes realistic ranges for:

- Altitude in km

- Velocity in km/s

- Inclination in degrees

- Type of satellite: LEO, MEO, GEO 

In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
orbital_types = {
    'LEO': {
        'altitude_km': (160, 2000),
        'velocity_kmps': (6.9, 8.0),
        'inclination_deg': (0, 180)
    },
    'MEO': {
        'altitude_km': (2000, 35785),
        'velocity_kmps': (3.0, 6.8),
        'inclination_deg': (0, 70)
    },
    'GEO': {
        'altitude_km': (35785, 35786),
        'velocity_kmps': (3.07, 3.09),
        'inclination_deg': (0, 5)
    }
}


In [ ]:
samples_per_type = {
    'LEO': 500,
    'MEO': 300,
    'GEO': 200
}


In [ ]:
# Generate synthetic data
data = []

for sat_type, count in samples_per_type.items():
    for _ in range(count):
        altitude = round(random.uniform(*orbital_types[sat_type]['altitude_km']), 2)
        velocity = round(random.uniform(*orbital_types[sat_type]['velocity_kmps']), 2)
        inclination = round(random.uniform(*orbital_types[sat_type]['inclination_deg']), 2)
        data.append([altitude, velocity, inclination, sat_type])


In [ ]:
# Convert to DataFrame
df = pd.DataFrame(data, columns=['altitude_km', 'velocity_kmps', 'inclination_deg', 'type_of_satellite'])

# Save as CSV
csv_path = "/Users/suparnac/dev_envs/CV_Projects/Orbital_Type_Classifier/satellite_orbital_classification_dataset.csv"
df.to_csv(csv_path, index=False)

csv_path

NOW, let's analyze this generated/Synthetic dataset which is an imbalance dataset 

In [ ]:
df = pd.read_csv('/Users/suparnac/dev_envs/CV_Projects/Orbital_Type_Classifier/satellite_orbital_classification_dataset.csv')
print(df.head(3))

In [ ]:
print(df['type_of_satellite'].value_counts())  # Check for imbalance

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Encode class labels LEo as 0, MEO as 1, and GEO as 2
df['type_encoded'] = df['type_of_satellite'].map({'LEO': 0, 'MEO': 1, 'GEO': 2})

X = df[['altitude_km', 'velocity_kmps', 'inclination_deg']]
y = df['type_encoded']

In [ ]:
# Split for train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:
# Scale
scaler = StandardScaler() #creates an instance of the StandardScaler. The StandardScaler standardizes features by removing the mean and scaling to unit variance.
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.svm import SVC

In [ ]:
#Handle Class Imbalance using Class Weights

model = SVC(class_weight='balanced')  # Automatically balances weights
model.fit(X_train_scaled, y_train)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
#evaluate performance


y_pred = model.predict(X_test_scaled)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['LEO', 'MEO', 'GEO']))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Visualize Class Separation

df['type_encoded'] = y
sns.pairplot(df, hue="type_of_satellite", palette="Set2")
plt.suptitle("Class Distribution by Features")
plt.show()


In [ ]:
#save the model
import joblib

joblib.dump(model, "orbit_classifier_model.pkl")
joblib.dump(scaler, "scaler.pkl")
